In [1]:
import pennylane as qml
import numpy as np
from collections import defaultdict

def entangling_capability(circuit_template, num_qubits, num_params, L, num_samples=1000):
    """Compute entangling capability (Q) of a parameterized quantum circuit."""
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def run_circuit(params, input_state):
        """Run the quantum circuit with given parameters."""
        qml.BasisState(input_state, wires=range(num_qubits))
        
        # Apply the circuit L times
        param_index = 0
        for _ in range(L):
            layer_params = params[param_index:param_index + num_params]
            circuit_template(layer_params, wires=range(num_qubits))
            param_index += num_params
        return qml.state()

    q_values = []
    total_params = num_params * L

    for _ in range(num_samples):
        params = np.random.uniform(0, 2 * np.pi, total_params)
        
        # Test all possible basis states
        for state_idx in range(2**num_qubits):
            input_state = [int(x) for x in np.binary_repr(state_idx, width=num_qubits)]
            state = run_circuit(params, input_state)

            # Meyer-Wallach Q calculation
            sum_purities = 0.0
            for qubit in range(num_qubits):
                psi = state.reshape([2] * num_qubits)
                psi = np.moveaxis(psi, qubit, 0).reshape(2, -1)

                rho = psi @ psi.conj().T
                purity = np.trace(rho @ rho).real
                sum_purities += purity

            Q = 2 * (1 - sum_purities / num_qubits)
            q_values.append(Q)

    return np.mean(q_values)

# Ansatz definitions
def U_TTN(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

def U_5(params, wires):
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRZ(params[4], wires=[wires[1], wires[0]])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])

def U_6(params, wires):
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])

def U_9(params, wires):
    qml.Hadamard(wires=wires[0])
    qml.Hadamard(wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])

def U_13(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRZ(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])

def U_14(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRX(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRX(params[5], wires=[wires[0], wires[1]])

def U_15(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

def U_SO4(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0])
    qml.RY(params[5], wires=wires[1])

def U_SU4(params, wires):
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])

# Circuit list configuration
circuit_list = [
    {"name": "U_TTN", "template": U_TTN, "num_qubits": 2, "num_params": 2},
    {"name": "U_5", "template": U_5, "num_qubits": 2, "num_params": 10},
    {"name": "U_6", "template": U_6, "num_qubits": 2, "num_params": 10},
    {"name": "U_9", "template": U_9, "num_qubits": 2, "num_params": 2},
    {"name": "U_13", "template": U_13, "num_qubits": 2, "num_params": 6},
    {"name": "U_14", "template": U_14, "num_qubits": 2, "num_params": 6},
    {"name": "U_15", "template": U_15, "num_qubits": 2, "num_params": 4},
    {"name": "U_SO4", "template": U_SO4, "num_qubits": 2, "num_params": 6},
    {"name": "U_SU4", "template": U_SU4, "num_qubits": 2, "num_params": 15},
]

def calculate_total_params(circuit, L):
    return circuit["num_params"] * L

if __name__ == "__main__":
    results = []
    layers_to_test = range(1, 6)  # Testing 1 to 5 layers
    
    print("Analyzing entangling capability...")
    print("Circuit\tLayers\tEntCap\tTotalParams")
    
    for circuit in circuit_list:
        for L in layers_to_test:
            Q = entangling_capability(
                circuit['template'],
                circuit['num_qubits'],
                circuit['num_params'],
                L,
                num_samples=100  # Reduced for faster testing
            )
            total_params = calculate_total_params(circuit, L)
            print(f"{circuit['name']}\t{L}\t{Q:.4f}\t{total_params}")
            results.append({
                'name': circuit['name'],
                'layers': L,
                'entangling_capability': Q,
                'total_params': total_params
            })

    # You can add further analysis/visualization of results here

Analyzing entangling capability...
Circuit	Layers	EntCap	TotalParams
U_TTN	1	0.2702	2
U_TTN	2	0.3044	4
U_TTN	3	0.3113	6
U_TTN	4	0.3461	8
U_TTN	5	0.3147	10
U_5	1	0.1342	10
U_5	2	0.2604	20
U_5	3	0.3531	30
U_5	4	0.3870	40
U_5	5	0.3934	50
U_6	1	0.3160	10
U_6	2	0.4112	20
U_6	3	0.3932	30
U_6	4	0.3988	40
U_6	5	0.3901	50
U_9	1	1.0000	2
U_9	2	0.0000	4
U_9	3	0.2642	6
U_9	4	0.5371	8
U_9	5	0.3535	10
U_13	1	0.2068	6
U_13	2	0.3335	12
U_13	3	0.3514	18
U_13	4	0.4205	24
U_13	5	0.3786	30
U_14	1	0.2237	6
U_14	2	0.3130	12
U_14	3	0.3779	18
U_14	4	0.3833	24
U_14	5	0.4231	30
U_15	1	0.3119	4
U_15	2	0.3384	8
U_15	3	0.3432	12
U_15	4	0.3062	16
U_15	5	0.3246	20
U_SO4	1	0.3405	6
U_SO4	2	0.3415	12
U_SO4	3	0.3699	18
U_SO4	4	0.3074	24
U_SO4	5	0.3241	30
U_SU4	1	0.3346	15
U_SU4	2	0.4217	30
U_SU4	3	0.3814	45
U_SU4	4	0.3775	60
U_SU4	5	0.4070	75
